#Load dataset (VoxCeleb1)

In [16]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import tarfile
file = tarfile.open('drive/MyDrive/Speech_understanding/VoxCeleb1_subset.tar-tron’s MacBook Pro.gz')

In [4]:
file.extractall('./VoxCeleb1')
file.close()

#Load Pre-trained Models

##ECAPA

In [13]:
!pip install git+https://github.com/speechbrain/speechbrain.git@develop

  Cloning https://github.com/speechbrain/speechbrain.git (to revision develop) to /tmp/pip-req-build-0asfbmio
  Running command git clone --filter=blob:none --quiet https://github.com/speechbrain/speechbrain.git /tmp/pip-req-build-0asfbmio
  Resolved https://github.com/speechbrain/speechbrain.git to commit ae51e2c509170f6937dd16892a4ff643fbccaa37
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [14]:
from speechbrain.inference.speaker import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

##Wavlm-base-sv


In [17]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from datasets import load_dataset
import torch

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [19]:
feature_extractor_wavlm = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model_wavlm = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/58.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/wavlm-base-sv were not used when initializing WavLMForXVector: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at microsoft/wavlm-base-sv and are newly initialized: ['wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

##Unispeech-SAT-large

In [20]:
from transformers import UniSpeechSatForXVector

In [21]:
feature_extractor_unispeech = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/unispeech-sat-base-sv')
model_unispeech = UniSpeechSatForXVector.from_pretrained('microsoft/unispeech-sat-base-sv')

preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/58.2k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/unispeech-sat-base-sv were not used when initializing UniSpeechSatForXVector: ['unispeech_sat.encoder.pos_conv_embed.conv.weight_g', 'unispeech_sat.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing UniSpeechSatForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing UniSpeechSatForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of UniSpeechSatForXVector were not initialized from the model checkpoint at microsoft/unispeech-sat-base-sv and are newly initialized: ['unispeech_sat.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'unispeech_sat.encoder.pos_conv_embed.conv.parametrizati

#Compute EER

In [22]:
import os
import torchaudio
import numpy as np

from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import roc_curve
from scipy.spatial.distance import cosine

In [23]:
def get_embedding(wv, sr, feature_extractor, model):
  wv = wv.squeeze()
  inputs = feature_extractor(wv, return_tensors="pt", sampling_rate=sr, padding=True)
  embeddings = model(**inputs).embeddings
  embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu()
  embeddings = embeddings.detach().numpy().flatten()
  return embeddings

def compute_speaker_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)

def get_similarity_score(wv1, wv2, sr1,sr2,feature_extractor, model):
  embedding1 = get_embedding(wv1, sr1, feature_extractor, model)
  embedding2 = get_embedding(wv2, sr2, feature_extractor, model)

  return compute_speaker_similarity(embedding1, embedding2)

In [24]:
def compute_eer(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    return eer

In [17]:
dataset_dir = "./VoxCeleb1/VoxCeleb1_subset"
test_dir = os.path.join(dataset_dir, "test")

with open(os.path.join(dataset_dir, 'list_test_hard.txt'), "r") as f:
    pairs = [line.strip().split() for line in f]

In [18]:
test_scores = defaultdict(list)
test_labels = []
for pair in tqdm(pairs):
    file1_path = os.path.join(test_dir, pair[1])
    file2_path = os.path.join(test_dir, pair[2])
    if os.path.exists(file1_path) and os.path.exists(file2_path):

      waveform_1, sr1 = torchaudio.load(file1_path)
      waveform_2, sr2 = torchaudio.load(file2_path)

      score_wavlm = get_similarity_score(waveform_1, waveform_2, sr1,sr2,
                                         feature_extractor_wavlm, model_wavlm)
      score_unispeech = get_similarity_score(waveform_1, waveform_2, sr1, sr2,
                                             feature_extractor_unispeech, model_unispeech)
      score_ecapa, _ = verification.verify_files(file1_path, file2_path)

      test_scores['wavlm'].append(score_wavlm)
      test_scores['unispeech'].append(score_unispeech)
      test_scores['ecapa'].append(score_ecapa)

      test_labels.append(int(pair[0]))

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 10000/10000 [22:22<00:00,  7.45it/s]


In [19]:
test_eer_wavlm = compute_eer(test_labels, test_scores['wavlm'])
test_eer_unispeech = compute_eer(test_labels, test_scores['unispeech'])
test_eer_ecapa = compute_eer(test_labels, test_scores['ecapa'])

In [20]:
print("EER for wavlm", test_eer_wavlm)
print("EER for unispeech", test_eer_unispeech)
print("EER for ecapa", test_eer_ecapa)

EER for wavlm 0.16666666666666666
EER for unispeech 0.16666666666666666
EER for ecapa 0.5


#INDICSUPERB


In [1]:
!wget https://indic-asr-public.objectstore.e2enetworks.net/indic-superb/kathbath/clean/testkn_audio.tar

--2024-04-07 10:57:52--  https://indic-asr-public.objectstore.e2enetworks.net/indic-superb/kathbath/clean/testkn_audio.tar
Resolving indic-asr-public.objectstore.e2enetworks.net (indic-asr-public.objectstore.e2enetworks.net)... 101.53.136.18, 164.52.206.155, 101.53.152.30, ...
Connecting to indic-asr-public.objectstore.e2enetworks.net (indic-asr-public.objectstore.e2enetworks.net)|101.53.136.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3587717120 (3.3G) [application/x-tar]
Saving to: ‘testkn_audio.tar’

testkn_audio.tar    100%[===================>]   3.34G  11.7MB/s    in 9m 40s  

2024-04-07 11:07:35 (5.90 MB/s) - ‘testkn_audio.tar’ saved [3587717120/3587717120]



In [2]:
!tar -xvf testkn_audio.tar

Streaming output truncated to the last 5000 lines.
kb_data_clean_m4a/bengali/test_known/audio/844424931519974-711-f.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424931526172-188-m.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424931299275-1038-m.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424930449249-99-m.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424932326182-960-m.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424930901024-647-f.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424930697798-1175-f.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424932372745-1005-f.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424932426046-1175-f.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424932336853-858-f.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424932458653-673-m.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424932409904-858-f.m4a
kb_data_clean_m4a/bengali/test_known/audio/844424931309648-1038-m.m4a
kb_data_clean_m4a/bengali/test_known/audio/84442

In [3]:
!git clone https://github.com/AI4Bharat/IndicSUPERB.git

Cloning into 'IndicSUPERB'...
remote: Enumerating objects: 606, done.
remote: Counting objects: 100% (606/606), done.
remote: Compressing objects: 100% (445/445), done.
remote: Total 606 (delta 147), reused 601 (delta 145), pack-reused 0
Receiving objects: 100% (606/606), 4.37 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [4]:
!python IndicSUPERB/utilities/structure.py kb_data_clean_m4a/ kb_data_clean_wav telugu

Copying Transcripts
0it [00:00, ?it/s]
100% 2410/2410 [05:50<00:00,  6.88it/s]


In [10]:
import os
import itertools
import random

In [34]:
def get_wav_files(folder):
    return [file for file in os.listdir(folder) if file.endswith('.wav')]

def generate_pairs(speakers):
    pairs = []
    for speaker in speakers:
        wav_files = get_wav_files(os.path.join(base_folder, speaker))
        same_pairs = random.sample(list(itertools.combinations(wav_files, 2)), num_same_pairs)
        for pair in same_pairs:
            pairs.append((1, os.path.join(base_folder, speaker, pair[0]), os.path.join(base_folder, speaker, pair[1])))
        other_speakers = [s for s in speakers if s != speaker]

        for _ in range(num_diff_pairs):
            other_speaker = random.choice(other_speakers)
            other_wav_files = get_wav_files(os.path.join(base_folder, other_speaker))
            wav_file = random.choice(wav_files)
            other_wav_file = random.choice(other_wav_files)
            pairs.append((0, os.path.join(base_folder, speaker, wav_file), os.path.join(base_folder, other_speaker, other_wav_file)))
    return pairs

def write_pairs_to_file(pairs):
    with open(output_file, 'w') as file:
        for pair in pairs:
            file.write(f"{pair[0]} {pair[1]} {pair[2]}\n")

In [43]:
base_folder = 'kb_data_clean_wav/telugu/test_known/audio/'
output_file = 'telugu_test_verification_pairs.txt'
num_same_pairs = 3
num_diff_pairs = 3

speakers = os.listdir(base_folder)
pairs = generate_pairs(speakers)
write_pairs_to_file(pairs)

In [44]:
with open(output_file, "r") as f:
    pairs = [line.strip().split() for line in f]

In [45]:
len(pairs)

120

In [46]:
test_scores = defaultdict(list)
test_labels = []
for pair in tqdm(pairs):
    file1_path = pair[1]
    file2_path = pair[2]
    if os.path.exists(file1_path) and os.path.exists(file2_path):

      waveform_1, sr1 = torchaudio.load(file1_path)
      waveform_2, sr2 = torchaudio.load(file2_path)

      score_wavlm = get_similarity_score(waveform_1, waveform_2, sr1,sr2,
                                         feature_extractor_wavlm, model_wavlm)
      score_unispeech = get_similarity_score(waveform_1, waveform_2, sr1, sr2,
                                             feature_extractor_unispeech, model_unispeech)
      score_ecapa, _ = verification.verify_files(file1_path, file2_path)

      test_scores['wavlm'].append(score_wavlm)
      test_scores['unispeech'].append(score_unispeech)
      test_scores['ecapa'].append(score_ecapa)

      test_labels.append(int(pair[0]))

  0%|          | 0/120 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 120/120 [21:12<00:00, 10.61s/it]


In [47]:
test_eer_wavlm = compute_eer(test_labels, test_scores['wavlm'])
test_eer_unispeech = compute_eer(test_labels, test_scores['unispeech'])
test_eer_ecapa = compute_eer(test_labels, test_scores['ecapa'])

In [48]:
print("EER for wavlm", test_eer_wavlm)
print("EER for unispeech", test_eer_unispeech)
print("EER for ecapa", test_eer_ecapa)

EER for wavlm 0.1
EER for unispeech 0.08333333333333333
EER for ecapa 0.06666666666666667
